In [68]:
#url 받아오고 에러처리

import requests
import time
def download(method, url, param=None, data=None, timeout=1, maxretries=3):
    
    try:
        resp = requests.request(method, url, params=param, data=data, headers={"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"})
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and maxretries>0:  #500대 에러일시 서버잘못이니까 재시도 메세지 띄워줌
            print(maxretries)
            time.sleep(timeout)
            resp = download(method, url, param, data, timeout, maxretries-1)
        print(e.response.status_code)
        print(e.response.reason)
    return resp
    


In [69]:
#http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard
from bs4 import BeautifulSoup

def parseURL(seed):
    html = download("get", seed)
    dom = BeautifulSoup(html.text, "lxml")
    return [requests.compat.urljoin(seed, _.find_parent()["href"]) 
            for _ in dom.select("font.list_title") 
            if _.find_parent() and _.find_parent().has_attr("href")]

In [80]:
url = "http://www.ppomppu.co.kr/zboard/zboard.php"
html = download("get", url, param ={"id" : "freeboard"})
dom = BeautifulSoup(html.text, "lxml")

seed = list()
seed.extend([requests.compat.urljoin(url, _["href"])
             for _ in dom.select("#page_list a")])
seen = list()
queue = list()

while seed:
    baseURL = seed.pop(0) 
    seen.append(baseURL)
    linkList = parseURL(baseURL)
    for link in linkList:
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue : {0}, Seen : {1}".format(len(queue), len(seen)))

Queue : 30, Seen : 1
Queue : 60, Seen : 2
Queue : 90, Seen : 3
Queue : 120, Seen : 4
Queue : 150, Seen : 5
Queue : 180, Seen : 6
Queue : 210, Seen : 7
Queue : 240, Seen : 8
Queue : 270, Seen : 9
Queue : 300, Seen : 10


In [81]:
def parseContent(url):
    html = download("get", url)
    dom = BeautifulSoup(html.text, "lxml")
    
    return {"title":
           dom.select_one("font.view_title2").text.strip(),
           "body":
           dom.select_one("td.board-contents").text.strip(),
           "comments":[_.text.strip()
                     for _ in dom.select("div[id^=comment]")]}

In [72]:
seen

['http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=2&divpage=1221',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=3&divpage=1221',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=4&divpage=1221',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=5&divpage=1221',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=6&divpage=1221',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=7&divpage=1221',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=8&divpage=1221',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=9&divpage=1221',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=10&divpage=1221',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=11&divpage=1221']

In [73]:
parseContent("http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=2&divpage=1221&no=6543184")

{'title': '정준영 카카오톡 대화 위법 수집 증거능력 없다',
 'body': '정준영 측 “카카오톡 대화 위법 수집, 증거능력 없다”\n\n\n\n\n\n\n[출처: 서울신문에서 제공하는 기사입니다.] https://www.seoul.co.kr/news/newsView.php?id=20190716500138&wlog_tag3=naver#csidxc65db6fe26090868fe498017ccaa6bc\xa0\n\xa0\n\xa0\n\n\n\n\n\xa0\n\xa0\n\xa0\n쥐새끼들ㅋㅋ',
 'comments': ['쑤레기들..귀신은 뭐하나요 이런놈들 안잡아가고']}

In [74]:
#DB

In [82]:
import sqlite3

con = sqlite3.connect("bot4.db")
cur = con.cursor()

cur.executescript("""
    DROP TABLE IF EXISTS table3;
    CREATE TABLE table3(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, 
        title TEXT NOT NULL,
        content TEXT NOT NULL,
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    );
    
    DROP TABLE IF EXISTS table4;
    CREATE TABLE table4(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        table3_id INTEGER NOT NULL,
        comment TEXT NOT NULL
    );
""")

con.commit()

In [83]:
while queue: #여긴항상트루
    baseURL = queue.pop(0)
    seen.append(baseURL)
    content = parseContent(baseURL)
    cur.execute("""
        INSERT INTO table3
        (title, content)
        VALUES(?,?)
    """, [content["title"], content["body"]])
    con.commit()
    
    cur.execute("""
        SELECT id 
        FROM table3
        WHERE title=? AND content=?;
    """,  [content["title"], content["body"]])
    table3_id = cur.fetchone()[0]
    
    for comment in content["comments"]:
        cur.execute("""
            INSERT INTO table4
            (table3_id, comment)
            VALUES(?, ?)
        """, [table3_id, comment])
        
        
    con.commit()

